In [1]:
#pip install python-steam-api
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import requests
import time
import pandas as pd
import numpy as np
import json

The following chunk runs a selenium browser to open up steams most played games and create a list of links to those game pages. 
Then it extracts the app_id of the game and appends it to our api link to create a list of api's with the app ids we are interested in. 

In [2]:
user_agent = "Microsoft Edge UA string:Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10136"
options = webdriver.EdgeOptions()
options.add_argument('user-agent={0}'.format(user_agent))
options.add_argument("--headless=new")

driver = webdriver.Edge()

url = "https://store.steampowered.com/charts/mostplayed/"

links = []
app_ids = []

def get_app_ids(url, rec=False):
    page = driver.get(url)
    response = requests.get(url, user_agent)
    print(response.status_code, url)


    time.sleep(2)

    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN) 

        
    soup = BeautifulSoup(driver.page_source, "lxml")

    chart = soup.find("div", class_ = "_1A7NagdRz58_o8HPHMa3eE").find("table", class_ = "_3arZn0BMPzyhcYNADe193m")

    for ls in chart.find_all("td"):
        try:  
            link = (ls.find("a", href = True)["href"])
            links.append(link)
            app_id = link.split('/')[4]
            app_ids.append(app_id)

        except Exception as e:   
            None

get_app_ids(url)
    
print(app_ids)


driver.quit()

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
	GetHandleVerifier [0x00007FF6330DA785+20677]
	simdutf::get_active_implementation [0x00007FF633036F30+492592]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF63332DC3A+1807418]
	(No symbol) [0x00007FF632E621B2]
	(No symbol) [0x00007FF632E5FDD7]
	(No symbol) [0x00007FF632EA3A99]
	(No symbol) [0x00007FF632E981C3]
	(No symbol) [0x00007FF632E6CC36]
	(No symbol) [0x00007FF632E6C0C4]
	(No symbol) [0x00007FF632E6CA63]
	(No symbol) [0x00007FF632F558DD]
	(No symbol) [0x00007FF632F61D42]
	(No symbol) [0x00007FF632F5A703]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6331B478A+262026]
	simdutf::get_active_implementation [0x00007FF633044661+547681]
	simdutf::get_active_implementation [0x00007FF63303D914+519700]
	simdutf::get_active_implementation [0x0

We now want to create a function to run all of the collected api links to retreive data from the api and place it into a dataframe. 

In [3]:
def get_game_details(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    data = response.json()

    if data[str(app_id)]['success']:
        game_data = data[str(app_id)]['data']
        name = (game_data['name'])
        description = (game_data['short_description'])
        developer = (game_data['developers'])
        publisher = (game_data['publishers'])
        if 'release_date' in game_data:
            date = game_data['release_date']['date']
        else:
            date = 'coming soon'
        if 'genres' in game_data: 
            genre = [genre['description'] for genre in game_data['genres']]
        else:
            None
        if 'price_overview' in game_data:
            price = game_data['price_overview']['final_formatted']
        else:
            price = 'Free' if game_data['is_free'] else 'price not available'

        return name, description, developer, price, publisher, date, genre
    else:
        return 'details not found'

In [5]:
data = []
names = []
descriptions = []
prices = []
controller_support_list = []
developers = []
publishers = []
date_release = []
genres = []


for app_id in app_ids:
    name, description, developer, price, publisher, date, genre = get_game_details(app_id)
    names.append(name)
    descriptions.append(description)
    developers.append(developer)
    prices.append(price)
    publishers.append(publisher)
    date_release.append(date)
    genres.append(genre)
   


In [6]:
print(names)


['Counter-Strike 2', 'Dota 2', 'PUBG: BATTLEGROUNDS', 'Marvel Rivals', 'Grand Theft Auto V', 'Path of Exile 2', 'Rust', 'Banana', 'Wallpaper Engine', 'Delta Force', 'Palworld', 'War Thunder', 'NARAKA: BLADEPOINT', 'Apex Legends™', 'Red Dead Redemption 2', "Baldur's Gate 3", 'Stardew Valley', 'EA SPORTS FC™ 25', 'Call of Duty®', 'Football Manager 2024', "Tom Clancy's Rainbow Six® Siege", 'Sid Meier’s Civilization® VI', 'Cyberpunk 2077', 'Warframe', 'Crosshair X', 'DayZ', 'ELDEN RING', 'Hearts of Iron IV', 'Euro Truck Simulator 2', 'Crab Game', 'The Sims™ 4', 'Project Zomboid', 'The Elder Scrolls V: Skyrim Special Edition', 'HELLDIVERS™ 2', 'Valheim', 'Team Fortress 2', 'VRChat', 'Farming Simulator 25', "No Man's Sky", 'Dead by Daylight', 'tModLoader', 'Monster Hunter: World', 'ARK: Survival Evolved', 'Terraria', '7 Days to Die', 'The Binding of Isaac: Rebirth', "Don't Starve Together", 'Black Myth: Wukong', 'Once Human', 'The Witcher 3: Wild Hunt', 'RimWorld', 'FINAL FANTASY VII REBIRTH

In [36]:
first_genre = []

for gre in genres:
    first_genre.append(gre[0])

first_genre
    

['Action',
 'Action',
 'Action',
 'Action',
 'Action',
 'Action',
 'Action',
 'Action',
 'Adventure',
 'Action',
 'Action',
 'Action',
 'Action',
 'Simulation',
 'Adventure',
 'Action',
 'Action',
 'Könnyed',
 'Action',
 'Action',
 'Simulation',
 '独立',
 'Action',
 'Indie',
 'Szimuláció',
 'Action',
 'Strategy',
 'Simulation',
 'RPG',
 'Action',
 'Indie',
 'Action',
 'Action',
 'Casual',
 'Action',
 'Action',
 'Adventure',
 'Action',
 'Adventure',
 'Action',
 'Action',
 'Action',
 'RPG',
 'Action',
 'Action',
 'Adventure',
 'Action',
 'Action',
 'Casual',
 'Action',
 'Action',
 'Action',
 'Action',
 'RPG',
 'Action',
 'Action',
 'Action',
 'Action',
 'Action',
 'Action',
 'Casual',
 'Action',
 'Racing',
 'Action',
 'Action',
 'Action',
 'Action',
 'Action',
 'Indie',
 'Casual',
 'Strategy',
 'RPG',
 'Action',
 'Massively Multiplayer',
 'Action',
 'Action',
 '模拟',
 'Audio Production',
 'Action',
 'Action',
 'Strategy',
 'Action',
 'Action',
 'Action',
 'Indie',
 'Action',
 'Action',
 'Ac

In [41]:
first_publisher = []

for pub in publishers: 
    first_publisher.append(pub[0])

first_publisher

['Valve',
 'Valve',
 'Grinding Gear Games',
 'NetEase Games',
 'KRAFTON, Inc.',
 'Facepunch Studios',
 'Rockstar Games',
 'PlayStation Publishing LLC',
 'Sky',
 'Gaijin Network Ltd',
 'TiMi Studio Group',
 'Activision',
 'CenterPoint Gaming',
 'Electronic Arts',
 'Larian Studios',
 'Digital Extremes',
 'Electronic Arts',
 'Wallpaper Engine Team',
 'Amazon Games',
 'Ubisoft',
 'SEGA',
 'SCS Software',
 'Bohemia Interactive',
 'ConcernedApe',
 'Paradox Interactive',
 'Valve',
 '2K',
 'Giants Software',
 'CD PROJEKT RED',
 'Dani',
 'The Indie Stone',
 'FromSoftware, Inc.',
 'Behaviour Interactive Inc.',
 'Wube Software LTD.',
 'NetEase Games Global',
 'SEGA',
 'Electronic Arts',
 'Rockstar Games',
 'VRChat Inc.',
 'GSC Game World (worldwide)',
 'Coffee Stain Publishing',
 'Endnight Games Ltd',
 'CD PROJEKT RED',
 'Nicalis, Inc.',
 'Psyonix LLC',
 'Coffee Stain Publishing',
 'Starry Studio',
 'Starbreeze Publishing AB',
 'Valve',
 'Kinetic Games',
 'Re-Logic',
 'Valve',
 'Pocketpair',
 'Be

In [50]:
import sqlite3

# define connection and cursor

connection = sqlite3.connect("steam_games_details.db")

cursor = connection.cursor()

# create games table

cursor.execute("""CREATE TABLE IF NOT EXISTS games (
               name text, 
               description text, 
               price text, 
               developers text, 
               publishers text, 
               date_release text, 
               genres text
               )""")

for n, d, p, de, pu, dr, g  in zip(names, descriptions, prices, first_dev, first_publisher, date_release, first_genre):
    cursor.execute('''INSERT OR REPLACE INTO games (name, description, price, developers, publishers, date_release, genres ) 
                   VALUES (?,?,?,?,?,?,?)''', 
                   (n, d, p, de, pu, dr, g)
                   )

cursor.execute("SELECT * FROM games")

x = cursor.fetchall()

connection.commit()

connection.close()

steam_games_df = pd.DataFrame(x)
steam_games_df.columns = [i[0] for i in cursor.description]
steam_games_df

,name,description,price,developers,publishers,date_release,genres
0,Counter-Strike 2,"For over two decades, Counter-Strike has offer...",Free,Valve,Valve,"21 Aug, 2012",Action
1,Dota 2,"Every day, millions of players worldwide enter...",Free,Valve,Valve,"Jul 9, 2013",Action
2,Path of Exile 2,Path of Exile 2 is a next generation free-to-p...,$29.99,Grinding Gear Games,Grinding Gear Games,"Dec 6, 2024",Action
3,Marvel Rivals,Marvel Rivals is a Super Hero Team-Based PVP S...,Free,NetEase Games,NetEase Games,"Dec 5, 2024",Action
4,PUBG: BATTLEGROUNDS,Play PUBG: BATTLEGROUNDS for free. Land on str...,Free,PUBG Corporation,"KRAFTON, Inc.","Dec 21, 2017",Action
...,...,...,...,...,...,...,...
95,eFootball™,The feverish football game with a worldwide to...,Free,KONAMI,KONAMI,"Sep 29, 2021",Simulation
96,World of Tanks Blitz,"Tanks, not war: jump into a free-to-play MMO a...",Free,Wargaming Group Limited,Wargaming Group Limited,"9 Nov, 2016",Action
97,Fallout 4,"Bethesda Game Studios, the award-winning creat...",$7.99,Bethesda Game Studios,Bethesda Softworks,"Nov 9, 2015",RPG
98,Brawlhalla,An epic platform fighter for up to 8 players o...,Free,Blue Mammoth Games,Ubisoft,"Oct 17, 2017",Action


In [49]:
connection = sqlite3.connect("steam_games_details.db")

cursor = connection.cursor()

cursor.execute("DROP TABLE games")

connection.commit()

connection.close()


Created a dump sql file to move table over to "steam_games_details.db" in postgressql 

In [51]:
connection = sqlite3.connect("steam_games_details.db")
cursor = connection.cursor()

with open('dump.sql', 'w') as f:
    for line in connection.iterdump():
        f.write('%s\n' % line)

connection.commit()
connection.close()

